In [59]:
import numpy as np
import matplotlib.pyplot as plt
# from scipy.signal import find_peaks

# Generate random data
x = np.linspace(0, 10, 100)
# y = np.random.randn(len(x))
# function to generate deterministic data
y = np.sin(x) * np.exp(-x / 2) + np.random.randn(len(x)) * 0.1

def default_temp_decay(temp):
    return temp * 0.999


def find_peaks_by_annealing(y, initial_temp: float = 1.0, final_temp: float = 0.01, temp_evo_op: callable = default_temp_decay) -> list[int]:
    temp = initial_temp

    # Random initial solution
    solution = np.random.randint(0, 2, len(y))

    counter = 0
    while temp > final_temp:
        # Evaluate current solution
        current_cost = np.sum(y[solution == 1])

        # Generate a new solution
        new_solution = solution.copy()
        idx = np.random.randint(0, len(y))
        new_solution[idx] = 1 - new_solution[idx]
        new_cost = np.sum(y[new_solution == 1])

        # Acceptance criterion
        if new_cost > current_cost:
            solution = new_solution
        else:
            if np.random.rand() < np.exp((new_cost - current_cost) / temp):
                solution = new_solution

        # Update temperature
        temp = temp_evo_op(temp)

        # perturb the temperature if the solution is not improving
        if new_cost <= current_cost:
            counter += 1
            if counter > 10:
                temp = temp * 1.01
                counter = 0

    return np.where(solution == 1)[0]


def smart_find_peaks_by_annealing(y, initial_temp: float = 1.0, final_temp: float = 0.01, temp_evo_op: callable = default_temp_decay) -> list[int]:
    """ The previous algorithm is not very smart. It just randomly selects a peak and tries to improve it. 
    
    This one is a bit smarter. It starts by finding the highest peak. Then, it tries to improve it. If it can't, it
    tries to improve the second highest peak. And so on.
    """

    temp = initial_temp

    # Random initial solution
    solution = np.random.randint(0, 2, len(y))

    counter = 0
    while temp > final_temp:
        # Evaluate current solution
        current_cost = np.sum(y[solution == 1])

        # Generate a new solution
        new_solution = solution.copy()
        idx = np.random.randint(0, len(y))
        new_solution[idx] = 1 - new_solution[idx]
        new_cost = np.sum(y[new_solution == 1])

        # Acceptance criterion
        if new_cost > current_cost:
            solution = new_solution
        else:
            if np.random.rand() < np.exp((new_cost - current_cost) / temp):
                solution = new_solution

        # Update temperature
        temp = temp_evo_op(temp)

        # perturb the temperature if the solution is not improving
        if new_cost <= current_cost:
            counter += 1
            if counter > 10:
                temp = temp * 1.01
                counter = 0

    return np.where(solution == 1)[0]

import numpy as np

def yet_smarter_find_peaks_by_annealing(y, initial_temp: float = 1.0, final_temp: float = 0.01, max_iterations: int = 10000000, temp_evo_op: callable = default_temp_decay) -> list[int]:
    temp = initial_temp
    stagnation_counter = 0
    solution = np.random.randint(0, 2, len(y))

    iteration = 0
    while temp > final_temp and iteration < max_iterations:
        current_cost = np.sum(y[solution == 1])
        new_solution = solution.copy()

        # Solution generation modified to flip more than one bit when the temperature is high
        for _ in range(int(temp * len(y))):
            idx = np.random.randint(0, len(y))
            if new_solution[idx] == 0 or np.random.rand() < 0.5:  # Here we bias towards adding more peaks
                new_solution[idx] = 1 - new_solution[idx]

        new_cost = np.sum(y[new_solution == 1])

        if new_cost > current_cost:
            solution = new_solution
            stagnation_counter = 0
        else:
            acceptance_probability = np.exp((new_cost - current_cost) / temp)
            if np.random.rand() < acceptance_probability:
                solution = new_solution
                stagnation_counter = 0
            else:
                stagnation_counter += 1

        temp = temp_evo_op(temp)

        # Perturbation strategy modified to increase temperature more significantly after long periods of stagnation
        if stagnation_counter > 10:
            temp = temp * (1 + 0.1 * stagnation_counter)
            if temp > initial_temp:
                temp = initial_temp
            stagnation_counter = 0

        iteration += 1

    return [i for i, val in enumerate(solution) if val == 1]  # Return indices of the peaks




peak_indices = yet_smarter_find_peaks_by_annealing(y)
# Plot the line chart
plt.figure(figsize=(10, 6))
plt.plot(x, y, label='Random Data', color='b')
plt.scatter(x[peak_indices], y[peak_indices], color='r', label='Local Maxima')

# Set labels and title
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Simulated Annealing Local maxima')

# Show legend
plt.legend()

# Show the plot
plt.show()



KeyboardInterrupt: 